In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mping 
import numpy as np
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import cv2

#Steps:            Parameters:
#1:Get image       (nothing)         
#2:Get Masked      (Thresholds)             
#3:Darken          (level of change) 
#4:Merge all       (nothing)         
#5:Convert to grey (nothing)  
#6:Blur            (kernel_size)
#7:Canny edge      (LowT,HighT)
#8:Area            (shape, size)
#9:Hough lines     (Threshold, min line length, max line gap, rho, theta)
#10:Draw           (Lines to ignore)

In [2]:
def ColorSelect(image, low_rgb, high_rgb):
    mask = np.copy(image)
    thresholds = (image[:,:,0] > high_rgb[0]) \
                   | (image[:,:,0] < low_rgb[0]) \
                   | (image[:,:,1] > high_rgb[1]) \
                   | (image[:,:,1] < low_rgb[1]) \
                   | (image[:,:,2] > high_rgb[2]) \
                   | (image[:,:,2] < low_rgb[2])   
                
    mask[thresholds] = [0,0,0]
    return mask

In [3]:
def RegionOfInterest(original, points):
    
    image = np.copy(original)
    mask = np.zeros_like(original)   
    ignore_mask_color = 255
    
    cv2.fillPoly(mask, points, ignore_mask_color)
    
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [20]:
def LaneLineDetector(original_image):
    #1
    image=np.copy(original_image) 
    resolution = image.shape
    
    #2
    yellow_low_thresholds = [180,160,0]
    yellow_high_thresholds = [255,255,140] 
    white_low_thresholds = [190,190,190]
    white_high_thresholds = [255,255,255]
                             
    white_mask = ColorSelect(image,white_low_thresholds,white_high_thresholds)
    yellow_mask = ColorSelect(image,yellow_low_thresholds,yellow_high_thresholds)

    #3 
    change = 100
    darker_image = np.where(image<change,0,image-change)

    #4                             
    whited_image = cv2.bitwise_or(darker_image, white_mask)
    masked_image = cv2.bitwise_or(whited_image, yellow_mask)
   
    #5                        
    grey_image = cv2.cvtColor(masked_image, cv2.COLOR_RGB2GRAY)
    
    #6
    kernel_size = 7                  
    blurred_image = cv2.GaussianBlur(grey_image,(kernel_size, kernel_size), 0)
    
    #7
    low_threshold = 50
    high_threshold = 150    
    edged_image = cv2.Canny(blurred_image, low_threshold, high_threshold)
 
    #8
    p1,p2,p3,p4  = [210,665],[600,430],[700,430],[1110,665]
    points = np.array([[p1,p2,p3,p4]], dtype=np.int32) 
    region_image= RegionOfInterest(edged_image, points)

    #9
    rho = 1
    theta = np.pi/180
    threshold = 20
    min_line_length = 50
    max_line_gap = 60
    lined_image = np.zeros_like(image)

    lines = cv2.HoughLinesP(region_image, rho, theta, threshold, np.array([]),
                                             min_line_length, max_line_gap)
    #10
    line_width = 5

    for line in lines:
        for x1,y1,x2,y2 in line: #No lines with little slope. No lines that cross the middle.
            if (np.absolute(y1-y2)>(np.absolute(x1-x2)*.5)):
                cv2.line(lined_image,(x1,y1),(x2,y2),(255,0,0),line_width)
                if (x1>690 and x2>690):
                    try:
                        cv2.line(lined_image,(rx,ry),(x1,y1),(0,0,255),line_width)
                    except:
                        pass
                    rx,ry=x2,y2
                elif (680>x1 and 680>x2):
                    try:
                        cv2.line(lined_image,(lx,ly),(x1,y1),(0,0,255),line_width)
                    except:
                        pass
                    lx,ly=x2,y2

    laned_image = cv2.addWeighted(image, 0.8, lined_image, 1, 0) 

    return laned_image 

In [21]:
final_clip_output = 'final_output1.mp4'
test_clip = VideoFileClip("test.mp4")
final_clip = test_clip.fl_image(LaneLineDetector)
%time final_clip.write_videofile(final_clip_output, audio=False)

[MoviePy] >>>> Building video final_output1.mp4
[MoviePy] Writing video final_output1.mp4



 62%|██████▏   | 156/251 [00:27<00:20,  4.63it/s]


100%|██████████| 251/251 [00:46<00:00,  5.21it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: final_output1.mp4 

CPU times: user 2min, sys: 8.51 s, total: 2min 9s
Wall time: 48.3 s


In [22]:
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(final_clip_output))

In [24]:
white_output = 'white_output1.mp4'
testw_clip = VideoFileClip("solidWhiteRight.mp4")
white_clip = testw_clip.fl_image(LaneLineDetector)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white_output1.mp4
[MoviePy] Writing video white_output1.mp4



100%|█████████▉| 221/222 [00:22<00:00,  8.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white_output1.mp4 

CPU times: user 1min 3s, sys: 4.28 s, total: 1min 8s
Wall time: 23.8 s


In [25]:
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(white_output))

In [26]:
yellow_output = 'yellow_output1.mp4'
testy_clip = VideoFileClip("solidYellowLeft.mp4")
yellow_clip = testy_clip.fl_image(LaneLineDetector)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow_output1.mp4
[MoviePy] Writing video yellow_output1.mp4



  1%|          | 4/682 [00:00<01:00, 11.27it/s]

TypeError: 'NoneType' object is not iterable

In [ ]:
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(yellow_output))